In [1]:
import numpy as np
import pandas as pd

import datetime
from datetime import datetime, date

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_rows', 30)
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 13)
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 서울
people = pd.read_csv('/content/drive/MyDrive/DS프로젝트_개인/최종/자치구단위 서울생활인구 일별 집계표.csv', encoding = 'cp949')

#people['기준일ID'] = pd.to_datetime(people['기준일ID'], format='%YYYY-%mm-%dd')
people['기준일ID']=pd.to_datetime(people['기준일ID'].astype(str), format='%Y/%m/%d')
df = people[['기준일ID', '시군구코드', '시군구명', '총생활인구수']]
df['year'] = df['기준일ID'].dt.year
df['month'] = df['기준일ID'].dt.month

df1 = df[(df['year'] == 2019) & (df['month'] == 7)]
df3 = df1.groupby(['시군구명']).agg({'총생활인구수':'mean'})
df2 = df[(df['year'] == 2019) & (df['month'] == 8)]
df4 = df2.groupby(['시군구명']).agg({'총생활인구수':'mean'})

population = (df3 + df4)/ 2
pop = population.reindex(index=['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구','강서구','구로구','금천구','영등포구','동작구','관악구','서초구','강남구','송파구','강동구', '서울시'])
pop = pop[0:25] / 1000

station = pd.read_excel('/content/drive/MyDrive/DS프로젝트_개인/최종/20220429기준_서울시정류소리스트.xlsx')
station.groupby(['ARS-자치구번호']).agg({'ARS-ID':'count'})

bus_stop = station.groupby(['ARS-자치구번호']).agg({'ARS-ID':'count'})
bus_stop.reset_index()
bus_stop.rename(index={1:'종로구', 2:'중구', 3: '용산구', 4: '성동구', 5: '광진구', 6: '동대문구', 7: '중랑구', 8:'성북구',
                        9: '강북구', 10: '도봉구', 11: '노원구', 12: '은평구', 13:'서대문구', 14: '마포구', 15: '양천구',16: '강서구',
                        17: '구로구', 18: '금천구', 19: '영등포구', 20: '동작구',21: '관악구', 22: '서초구',23: '강남구',24: '송파구',
                        25: '강동구'}, inplace = True)

route = pd.read_excel('/content/drive/MyDrive/DS프로젝트_개인/최종/20220420기준_서울시_버스노선별정류소정보.xlsx')
route.groupby(['자치구번호']).agg({'노선ID': pd.Series.nunique}).head(30)
route_num = route.groupby(['자치구번호']).agg({'노선ID': pd.Series.nunique})[1:26]
route_num.reset_index()
route_num.rename(index={1:'종로구', 2:'중구', 3: '용산구', 4: '성동구', 5: '광진구', 6: '동대문구', 7: '중랑구', 8:'성북구',
                        9: '강북구', 10: '도봉구', 11: '노원구', 12: '은평구', 13:'서대문구', 14: '마포구', 15: '양천구',16: '강서구',
                        17: '구로구', 18: '금천구', 19: '영등포구', 20: '동작구',21: '관악구', 22: '서초구',23: '강남구',24: '송파구',
                        25: '강동구'}, inplace = True)

pop['버스정류장 수'] = bus_stop
pop['노선 수'] = route_num

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# 부산
before = pd.read_csv('/content/drive/MyDrive/DS프로젝트_개인/최종/노선별 부산버정.csv', encoding='cp949')
before = before.loc[:, ['BUSLINENUM', 'SIGUNNAME', 'GUNAME', 'X', 'Y', 'BUSSTOPID']]

nocity_before = before[before['GUNAME'].isnull()]
nocity = nocity_before[['X', 'Y']]
train_before = before.dropna(subset=['GUNAME'])
train = train_before[['X','Y']]
target = train_before['GUNAME']

classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(train, target)
pred = classifier.predict(nocity)
pred_col = pd.Series(pred, index=nocity.index)
whole_pred = pd.Series(pred_col, index=list(range(0, 24012)))

before['GUNAMES'] = np.where(pd.isnull(before['GUNAME']), whole_pred, before['GUNAME'])
before.drop_duplicates(['BUSSTOPID'], inplace=True)

info = ['기장군', '금정구', '해운대구', '수영구', '남구', '동구', '중구', '영도구', '강서구', '북구', '사상구', '사하구', '동래구', '부산진구', '연제구', '서구']
total = pd.Series(before['GUNAMES'].value_counts(), index=info)
un = [218.32, 65.28, 51.54, 10.22, 26.82, 9.87, 2.83, 14.2, 181.5, 39.37, 36.1, 41.77, 16.63, 29.67, 12.10, 13.95]
area = pd.Series(un, index = info)

sett = np.array([166756, 245489, 457402, 179958, 266520, 103246, 82810, 123586, 136115, 276922, 249460, 321592, 266507, 403883, 213763, 109185])
move = pd.Series(sett, index=info)/1000

lines = before[['BUSLINENUM', 'GUNAMES']]
whole_line = lines.groupby('GUNAMES')['BUSLINENUM'].unique()
whole_line = whole_line[info]
for i in info:
  whole_line[i] = len(whole_line[i])

result = pd.concat([total, area, move, whole_line], axis=1)
result.columns = ['STOPNUM', 'AREA', 'MOVE', 'BUSNUM']

result['DS'] = result['STOPNUM']/result['AREA']
result['DP'] = result['STOPNUM']/result['MOVE']
AA = result[['DS', 'DP']]
ds = (np.mean(result['DS']))
dp = (np.mean(result['DP']))
result['T'] = result['BUSNUM']/result['MOVE']

E = np.sqrt(np.sum(np.square(result['T'] - np.mean(result['T'])) / 15))
print('E =', E)
print(result)

E = 0.0728341178881734
      STOPNUM    AREA     MOVE BUSNUM         DS        DP         T
기장군       814  218.32  166.756     41   3.728472  4.881384  0.245868
금정구       499   65.28  245.489     37   7.643995  2.032678   0.15072
해운대구      620   51.54  457.402     43  12.029492  1.355482  0.094009
수영구       243   10.22  179.958     21  23.776908  1.350315  0.116694
남구        527   26.82  266.520     28  19.649515  1.977338  0.105058
동구        250    9.87  103.246     28  25.329281  2.421401  0.271197
중구        129    2.83   82.810     21  45.583039  1.557783  0.253593
영도구       405   14.20  123.586     19  28.521127  3.277070  0.153739
강서구      1124  181.50  136.115     37   6.192837  8.257723  0.271829
북구        454   39.37  276.922     36  11.531623  1.639451  0.130001
사상구       483   36.10  249.460     35  13.379501  1.936182  0.140303
사하구       638   41.77  321.592     38  15.274120  1.983880  0.118162
동래구       574   16.63  266.507     46  34.515935  2.153790  0.172603
부산진구      6

In [ ]:
# 제주
path1='/content/gdrive/MyDrive/Colab Notebooks/MachineLearning/Univ/DScover/Project/제주특별자치도_버스정류소현황_20170831.csv'
path2='/content/gdrive/MyDrive/Colab Notebooks/MachineLearning/Univ/DScover/Project/제주특별자치도_주제8_유동인구 비율 대비 지역별 인기있는 업종 추천 데이터_매쉬업결과_18.csv'
df_bus = pd.read_csv(path1, encoding='cp949') # 버스 현황
df_pop = pd.read_csv(path2, encoding='cp949') # 19년도 방문인구

df_bus= df_bus.drop(['경도','위도','위치정보(주변설명)','데이터기준일자'], axis=1)
df_bus['정류소ID'] = df_bus['정류소ID'].apply(str)
df_bus['정류소ID'] = df_bus['정류소ID'].str[:5]

df_jeju_bus = df_bus[df_bus['정류소ID']=='40500']
df_seogwipo_bus = df_bus[df_bus['정류소ID']=='40600']
df_jeju_bus_num = len(df_jeju_bus)
df_seogwipo_bus_num = len(df_seogwipo_bus)

df_pop= df_pop.drop(['읍면동명','업종명','이용자수','이용금액','데이터기준일자'], axis=1)
df_pop['월'] = df_pop['년월'].str.replace('-','')
df_pop = df_pop.drop('년월',axis=1)
df_pop['월'] = df_pop['월'].str[4:6]
df_pop = df_pop.drop_duplicates()

df_pop = df_pop[(df_pop['월']>='07') &(df_pop['월']<='08')]
df_jeju_pop = df_pop[df_pop['시도명']=='제주시']
df_seogwipo_pop = df_pop[df_pop['시도명']=='서귀포시']
jeju_pop = (df_jeju_pop[['월','총 유동인구']].groupby('월').sum().mean())/30000
seogwipo_pop = (df_seogwipo_pop[['월','총 유동인구']].groupby('월').sum().mean())/30000

jeju_area = 977.8
seogwipo_area = 870.7
jeju_ds = df_jeju_bus_num/jeju_area
seogwipo_ds = df_seogwipo_bus_num/seogwipo_area
ds = (jeju_ds + seogwipo_ds) / 2

jeju_dp = df_jeju_bus_num/jeju_pop
seogwipo_dp = df_seogwipo_bus_num/seogwipo_pop
dp = (jeju_dp + seogwipo_dp) / 2

print('제주도 Ds:',ds)
print('제주도 DP:',dp)